### Load in the training data

In [2]:
import pandas as pd
import numpy as np

# # Use read_excel and use participant_id as the index
# train_cat = pd.read_excel(
#     "../../widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
# train_func = pd.read_csv("../../widsdatathon2025/TRAIN_NEW/"
#                          "TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
# train_quant = pd.read_excel(
#     "../../widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
# train_soln = pd.read_excel(
#     "../../widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

# Use read_excel and use participant_id as the index
train_cat = pd.read_excel("../data/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_func = pd.read_csv("../data/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_quant = pd.read_excel("../data/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_soln = pd.read_excel("../data/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

In [3]:
# Set index for merging
train_cat.set_index("participant_id", inplace=True)
train_quant.set_index("participant_id", inplace=True)
train_func.set_index("participant_id", inplace=True)
train_soln.set_index("participant_id", inplace=True)

# train_cat.head()
train_func.head()
# train_quant.head()
# train_soln.head()

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,0.470063,...,0.224985,0.397448,0.422966,0.184642,0.305549,0.420349,0.016328,0.561864,0.471170,0.365221
WHWymJu6zNZi,0.614765,0.577255,0.496127,0.496606,0.404686,0.439724,0.122590,-0.085452,0.120673,0.276350,...,0.217546,-0.014549,0.000440,-0.096451,0.454501,0.343916,0.167313,0.607656,0.550623,0.503176
4PAQp1M6EyAo,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,0.378557,...,0.342487,-0.021141,-0.037836,0.075069,0.412712,0.292708,0.391005,0.461544,0.508912,0.624232
obEacy4Of68I,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,0.259596,...,0.103562,-0.178313,0.210983,-0.018666,0.436313,0.592982,0.216205,0.341272,0.440313,0.558193
s7WzzDcmDOhF,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,0.331445,...,-0.164956,0.007064,-0.120904,-0.488095,0.493575,-0.215361,0.210685,0.055850,0.119065,0.108273


## PCA for the functional connectome matrix to reduce dimension

In [4]:
from sklearn.decomposition import PCA

# Select only fMRI columns
fmri_data = train_func.drop(columns=['participant_id'], errors='ignore')

# Apply PCA to reduce to top N components (e.g. 50)
pca = PCA(n_components=50)
fmri_pca = pca.fit_transform(fmri_data)

train_func_pca = pd.DataFrame(fmri_pca, index=train_func.index,
                           columns=[f'fmri_pca_{i}' for i in range(1, 51)])
train_func_pca.head()

,fmri_pca_1,fmri_pca_2,fmri_pca_3,fmri_pca_4,fmri_pca_5,fmri_pca_6,fmri_pca_7,fmri_pca_8,fmri_pca_9,fmri_pca_10,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,6.322800,-0.365680,-4.918161,-3.823233,-5.706919,-0.875163,1.567532,-1.347492,-1.717385,1.626795,...,0.426761,0.590160,-0.098446,-0.573865,1.524296,-4.512346,2.622819,0.327805,1.244732,1.677228
WHWymJu6zNZi,5.468294,-3.985252,1.542399,0.104718,-0.529035,-2.951973,0.786363,0.794948,-2.772042,-2.408829,...,1.425251,-0.161658,-0.786797,0.103781,-1.848521,-0.111205,-0.951117,-0.754165,-1.027301,-0.745949
4PAQp1M6EyAo,0.447190,0.733000,3.051331,1.763902,1.932820,6.451159,-3.356866,1.946045,-1.165648,-2.876858,...,-2.048562,-0.534946,2.175221,0.540604,-0.421141,-1.952870,-0.060651,0.253094,-1.184290,-0.451805
obEacy4Of68I,-9.149799,-2.044626,0.519359,-2.703923,-4.444126,2.916932,1.036392,0.080346,-0.148158,0.384110,...,-0.629536,1.388507,0.539909,-0.724567,-1.558465,-0.344006,0.154312,-0.406017,-0.497200,0.795777
s7WzzDcmDOhF,0.812814,-1.933265,1.834524,0.383160,-0.512340,2.972221,3.151072,3.419826,0.283703,-2.997927,...,2.106058,2.781364,-2.022900,-0.606118,-0.364447,0.936331,-1.616465,2.252800,-2.057176,-0.467726


## Interpolate missing values with mean for quant and cat dfs

In [5]:
# Check missing values *before* merging
missing = train_quant.isnull().sum()
missing_cat = train_cat.isnull().sum()
print("Missing values in train_quant:\n", missing[missing > 0])
print("Missing values in train_cat:\n", missing_cat[missing_cat > 0])

# Define columns with small gaps to impute
small_gap_cols = [
    'EHQ_EHQ_Total', 'ColorVision_CV_Score',
    'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV',
    'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP',
    'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
    'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
    'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
    'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
    'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
    'MRI_Track_Scan_Location'
]

small_gap_cols_cat = [
    'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ'
]

# Fill small gaps using column means
for col in small_gap_cols:
    if col in train_quant.columns:
        train_quant[col] = train_quant[col].fillna(train_quant[col].mean())
for col in small_gap_cols_cat:
    if col in train_cat.columns:
        train_cat[col] = train_cat[col].fillna(train_cat[col].mean())

train_quant.drop(columns=['MRI_Track_Age_at_Scan'], inplace=True) # Dropped
train_cat.drop(columns=['Barratt_Barratt_P2_Occ', 'Barratt_Barratt_P2_Edu'], 
              inplace=True) # Dropped

Missing values in train_quant:
 EHQ_EHQ_Total                  13
ColorVision_CV_Score           23
APQ_P_APQ_P_CP                 12
APQ_P_APQ_P_ID                 12
APQ_P_APQ_P_INV                12
APQ_P_APQ_P_OPD                12
APQ_P_APQ_P_PM                 12
APQ_P_APQ_P_PP                 12
SDQ_SDQ_Conduct_Problems        9
SDQ_SDQ_Difficulties_Total      9
SDQ_SDQ_Emotional_Problems      9
SDQ_SDQ_Externalizing           9
SDQ_SDQ_Generating_Impact       9
SDQ_SDQ_Hyperactivity           9
SDQ_SDQ_Internalizing           9
SDQ_SDQ_Peer_Problems           9
SDQ_SDQ_Prosocial               9
MRI_Track_Age_at_Scan         360
dtype: int64
Missing values in train_cat:
 PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
dtype: int64


In [6]:
# Check missing values *before* merging
missing = train_quant.isnull().sum()
print("Missing values in train_quant:\n", missing[missing > 0])

Missing values in train_quant:
 Series([], dtype: int64)


In [7]:
train_quant.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
participant_id,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0
00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0
04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0
05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0
06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0


## Feature Selection for train_quant using random forests

In [8]:
# Make sure participant_id is a column, not the index
train_quant = train_quant.reset_index()
train_cat = train_cat.reset_index()
train_soln = train_soln.reset_index()
train_quant.head()

,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
0,00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0
1,00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0
2,04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0
3,05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0
4,06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0


In [9]:
train_cat.head()

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ
0,00aIpNTbG5uh,2019,4,1.0,0.0,3.0,21.0,45.0
1,00fV0OyyoLfw,2017,1,0.0,9.0,2.0,21.0,0.0
2,04X1eiS79T4B,2017,1,1.0,2.0,2.0,9.0,0.0
3,05ocQutkURd6,2018,1,3.0,8.0,2.0,18.0,10.0
4,06YUNBA9ZRLq,2018,1,0.0,1.0,2.0,12.0,0.0


In [10]:
# Combine with targets temporarily
quant_with_targets = train_quant.merge(train_soln[['participant_id', 'ADHD_Outcome',
                                                   'Sex_F']], on='participant_id')

# Compute correlations
corrs = quant_with_targets.drop(columns='participant_id').corr()

# Grab correlation with targets
adhd_corr = corrs['ADHD_Outcome'].abs().sort_values(ascending=False)
sex_corr = corrs['Sex_F'].abs().sort_values(ascending=False)

# Keep top N or those above a threshold
top_adhd_feats = adhd_corr[adhd_corr > 0.05].index.tolist()
top_sex_feats = sex_corr[sex_corr > 0.05].index.tolist()

# Union of both sets, drop targets themselves if included
quant_selected = list(set(top_adhd_feats + top_sex_feats) - {'ADHD_Outcome', 'Sex_F'})
quant_selected

['APQ_P_APQ_P_OPD',
 'SDQ_SDQ_Generating_Impact',
 'SDQ_SDQ_Emotional_Problems',
 'SDQ_SDQ_Prosocial',
 'APQ_P_APQ_P_ID',
 'APQ_P_APQ_P_CP',
 'SDQ_SDQ_Hyperactivity',
 'SDQ_SDQ_Conduct_Problems',
 'SDQ_SDQ_Peer_Problems',
 'APQ_P_APQ_P_PP',
 'APQ_P_APQ_P_INV',
 'SDQ_SDQ_Internalizing',
 'ColorVision_CV_Score',
 'SDQ_SDQ_Externalizing',
 'SDQ_SDQ_Difficulties_Total']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Merge everything on participant_id
X_full = train_quant[['participant_id'] + quant_selected] \
    .merge(train_cat, on='participant_id') \
    .merge(train_func_pca, on='participant_id')

# Target labels
y = train_soln[['participant_id', 'ADHD_Outcome', 'Sex_F']]

# Align rows across X and y
X_full = X_full.merge(y, on='participant_id')
y = X_full[['ADHD_Outcome', 'Sex_F']]
X_full = X_full.drop(columns=['ADHD_Outcome', 'Sex_F'])

# Deal with NaN values by filling with mean of the column
X_full = X_full.fillna(X_full.drop("participant_id", axis=1).mean())

# Use combined target just for ranking features
combo_target = y['ADHD_Outcome'] + y['Sex_F']

# Train the Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_full.drop(columns=['participant_id']), combo_target)

# Rank feature importances
feat_importances = pd.Series(rf.feature_importances_, index=X_full.drop(columns=['participant_id']).columns)
top_features_final = feat_importances.sort_values(ascending=False).head(50).index.tolist()

# Final X for model training
X_selected = X_full[top_features_final]
X_selected

,fmri_pca_13,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Externalizing,fmri_pca_14,fmri_pca_34,fmri_pca_26,SDQ_SDQ_Generating_Impact,fmri_pca_50,fmri_pca_35,...,fmri_pca_18,fmri_pca_17,fmri_pca_39,fmri_pca_10,fmri_pca_49,APQ_P_APQ_P_INV,fmri_pca_29,fmri_pca_47,fmri_pca_27,fmri_pca_38
0,0.968606,8.0,17.0,11.0,-3.181886,-0.066935,-0.257535,5.0,-3.605164,2.188987,...,2.375539,-1.155388,-6.158416,2.042070,1.065182,44.0,0.082271,-1.397553,1.181310,1.759896
1,0.667946,8.0,20.0,13.0,-0.274095,-1.425260,2.247302,5.0,-2.332640,2.210636,...,0.106528,-0.788108,-0.465582,-1.207644,-0.084868,35.0,-0.985647,2.993240,0.727344,1.051638
2,3.276908,7.0,24.0,10.0,3.505331,0.082102,-0.210387,10.0,-0.426936,-0.495404,...,-0.041774,-4.886835,-0.268875,-0.556097,-1.590623,37.0,-2.261313,0.278586,0.254056,0.402359
3,3.960095,3.0,5.0,3.0,1.134805,0.745797,0.873372,0.0,0.559102,1.051786,...,-1.052257,-0.753086,3.974851,2.460532,2.024152,42.0,1.971685,-0.817285,-0.147041,-1.617635
4,-0.961569,9.0,23.0,15.0,0.735015,-0.395874,-1.833026,8.0,0.142465,1.585534,...,1.292082,-2.317038,0.220579,1.825339,1.545337,35.0,0.305756,1.900645,1.120219,-0.093675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,-3.307589,8.0,20.0,13.0,-2.747729,1.081543,1.009799,5.0,0.908502,-0.829783,...,1.234600,1.343857,0.783379,-0.851544,-1.118779,36.0,-2.408755,-0.565889,1.046826,0.849766
1209,-2.256020,3.0,5.0,5.0,-2.002874,0.497511,-0.961492,0.0,-0.781181,0.699206,...,-0.550545,-0.699994,1.459967,-2.511263,0.108973,42.0,0.171723,-0.194557,-0.293773,-0.421084
1210,3.354588,7.0,14.0,9.0,0.470419,-1.852878,-0.445292,3.0,2.347286,1.622081,...,1.845323,0.979393,0.306977,1.253573,0.131524,34.0,-0.604173,2.031235,0.348662,-0.172969
1211,-1.665823,7.0,10.0,8.0,-1.334573,1.706057,2.089490,5.0,-0.981200,1.064918,...,0.086969,0.776179,-0.428524,0.192352,-0.539063,45.0,0.733245,1.045482,-0.347714,0.642238


In [17]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y[['ADHD_Outcome', 'Sex_F']].values, dtype=torch.float32)

# Stratified train/test split
stratify_labels = y['ADHD_Outcome'].astype(str) + y['Sex_F'].astype(str)
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=stratify_labels)

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

# Beefed-up model
class BetterMultiOutputNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.head_adhd = nn.Linear(128, 1)
        self.head_sex = nn.Linear(128, 1)

    def forward(self, x):
        shared_out = self.shared(x)
        return torch.sigmoid(self.head_adhd(shared_out)), torch.sigmoid(self.head_sex(shared_out))

# Weighted BCE Loss with 2x weight for Female ADHD cases
class WeightedBCELoss(nn.Module):
    def __init__(self, weight_female_adhd=2):
        super(WeightedBCELoss, self).__init__()
        self.weight_female_adhd = weight_female_adhd

    def forward(self, predictions, targets, sex):
        # Calculate standard BCE loss
        bce_loss = nn.BCEWithLogitsLoss(reduction='none')(predictions, targets)
        
        # Apply weight for female ADHD cases (ADHD_Outcome=1, Sex_F=1)
        weight = torch.ones_like(bce_loss)
        weight[(targets == 1) & (sex == 1)] = self.weight_female_adhd
        
        # Apply the weights to the loss
        weighted_loss = weight * bce_loss
        return weighted_loss.mean()

# Initialize model and loss function
model = BetterMultiOutputNN(X_selected.shape[1])
weighted_criterion = WeightedBCELoss(weight_female_adhd=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

# Training loop with early stopping
best_val_loss = np.inf
patience = 5
epochs_no_improve = 0

for epoch in range(30):
    model.train()
    running_loss = 0
    for xb, yb in train_loader:
        adhd_pred, sex_pred = model(xb)
        # Extract ADHD and Sex labels
        adhd_labels = yb[:, 0]
        sex_labels = yb[:, 1]
        
        # Compute weighted loss for both ADHD and Sex
        loss_adhd = weighted_criterion(adhd_pred.squeeze(), adhd_labels, sex_labels)
        loss_sex = weighted_criterion(sex_pred.squeeze(), sex_labels, sex_labels)
        
        # Total loss
        loss = loss_adhd + loss_sex
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for xb, yb in test_loader:
            adhd_pred, sex_pred = model(xb)
            loss_adhd = weighted_criterion(adhd_pred.squeeze(), yb[:, 0], yb[:, 1])
            loss_sex = weighted_criterion(sex_pred.squeeze(), yb[:, 1], yb[:, 1])
            val_loss = loss_adhd + loss_sex
            val_losses.append(val_loss.item())
    avg_val_loss = np.mean(val_losses)
    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch+1} | Train Loss: {running_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

# Evaluation
model.load_state_dict(best_model)
model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        adhd_pred, sex_pred = model(xb)
        preds = torch.cat([adhd_pred, sex_pred], dim=1)
        all_preds.append(preds)
        all_true.append(yb)

all_preds = torch.cat(all_preds).numpy()
all_true = torch.cat(all_true).numpy()

# Thresholding
adhd_pred_labels = (all_preds[:, 0] > 0.5).astype(int)
sex_pred_labels = (all_preds[:, 1] > 0.5).astype(int)

# Ensure the true labels are correctly handled (indexing properly)
all_true_adhd = all_true[:, 0].astype(int)
all_true_sex = all_true[:, 1].astype(int)

# Compute weighted F1 Score with 2x weight for Female ADHD cases
def weighted_f1(y_true, y_pred, weight_column):
    # Assign a weight of 2x for Female ADHD cases
    weight = np.ones_like(y_true)
    weight[(y_true == 1) & (weight_column == 1)] = 3  # Apply 2x weight for Female ADHD
    
    f1 = f1_score(y_true, y_pred, average='weighted', sample_weight=weight)
    return f1

f1_adhd = weighted_f1(all_true_adhd, adhd_pred_labels, all_true_sex)
f1_sex = weighted_f1(all_true_sex, sex_pred_labels, all_true_sex)

print(f"\nADHD Weighted F1 Score: {f1_adhd:.4f}")
print(f"Sex Weighted F1 Score: {f1_sex:.4f}")

# Metrics for ADHD
def print_metrics(y_true, y_pred, name):
    print(f"\n {name} Metrics:")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"F1 Score:  {f1_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred):.4f}")

print_metrics(all_true_adhd, adhd_pred_labels, "ADHD")
print_metrics(all_true_sex, sex_pred_labels, "Sex")

Epoch 1 | Train Loss: 50.4119 | Val Loss: 1.5845
Epoch 2 | Train Loss: 48.1687 | Val Loss: 1.5463
Epoch 3 | Train Loss: 47.2422 | Val Loss: 1.5354
Epoch 4 | Train Loss: 46.8948 | Val Loss: 1.5365
Epoch 5 | Train Loss: 45.7388 | Val Loss: 1.5245
Epoch 6 | Train Loss: 45.1903 | Val Loss: 1.5306
Epoch 7 | Train Loss: 45.0625 | Val Loss: 1.5237
Epoch 8 | Train Loss: 44.6962 | Val Loss: 1.5222
Epoch 9 | Train Loss: 44.5873 | Val Loss: 1.5301
Epoch 10 | Train Loss: 44.3154 | Val Loss: 1.5148
Epoch 11 | Train Loss: 43.9771 | Val Loss: 1.5201
Epoch 12 | Train Loss: 43.6940 | Val Loss: 1.5178
Epoch 13 | Train Loss: 43.8213 | Val Loss: 1.5297
Epoch 14 | Train Loss: 43.3669 | Val Loss: 1.5225
Epoch 15 | Train Loss: 43.2036 | Val Loss: 1.5219
Early stopping triggered.

ADHD Weighted F1 Score: 0.8160
Sex Weighted F1 Score: 0.5033

 ADHD Metrics:
Accuracy:  0.7737
F1 Score:  0.8442
Precision: 0.8011
Recall:    0.8922

 Sex Metrics:
Accuracy:  0.6461
F1 Score:  0.3944
Precision: 0.4746
Recall:    0.3